In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
import time

In [34]:
URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

response = requests.get(URL)

In [41]:
state = re.search(
    r'[A-Z][a-z]+(?: [A-Z][a-hj-z][a-z]+)?', 
    BS(response.text).findAll('title')[0].text 
)[0]
type(state)

str

In [36]:
district = re.search(
    r'District [0-9]{2}', 
    BS(response.text).findAll('title')[0].text
)[0]
type(district)

str

In [37]:
state_dict = {
    'State' : [state],
    'District' : [district]
}

In [38]:
pd.DataFrame.from_dict(state_dict)

,State,District
0,Tennessee,District 07


In [14]:
def district_scraper(state, district):
    URL = f'https://www.opensecrets.org/races/candidates?cycle=2020&id={state}{district}&spec=N'
    soup = BS(requests.get(URL).text)
    
    candidate_list = [x.text.strip() for x in soup.find_all('strong')]
    finances = (pd
        .read_html(
            str(soup.findAll('table', attrs={'class' : 'Members--table'}))
                .replace('$','')
                .replace(':', '')
        )
    )
    
    
    temp_df = pd.DataFrame()
    
    temp_df['name'] = [re.findall(r'(.+ .+) \(\w', item)[0] for item in candidate_list]
    
    temp_df['party'] = [re.findall(r'\((\w)\)', item)[0] for item in candidate_list]
    
    temp_df['state'] = str(re.search(r'[A-Z][a-z]+(?: [A-Z][a-z][a-rt-z][a-z]+)?',
                                     soup.findAll('title')[0].text)[0])
    
    temp_df['district'] = str(re.findall(r'District ([0-9]{2})', soup.findAll('title')[0].text)[0])
    
    temp_df['incumbent'] = ['Incumbent' if re.search(r'(Incumbent)', str(item)) != None 
                            else 'Not Incumbent' for item in candidate_list]
    
    temp_df['winner'] = ['Winner' if re.search(r'(Winner)', str(item)) is not None
                              else 'Not Winner' for item in candidate_list]
    
    temp_df['vote'] = [re.findall(r'(\d*[.?]\d*\%)', item)[0] for item in candidate_list]
    
    temp_df['raised'] = [finances[i].iloc[0,1] for i in range(len(candidate_list))]
    
    temp_df['spent'] = [finances[i].iloc[1,1] for i in range(len(candidate_list))]
    
    temp_df['cash_on_hand'] = [finances[i].iloc[2,1] for i in range(len(candidate_list))]
    
    global candidate_df
    candidate_df = pd.concat([candidate_df, temp_df])
    candidate_df = candidate_df.reset_index(drop=True)

In [15]:
district_dict = {
'AL': 7, 'AK': 1, 'AZ': 9, 'AR': 4, 'CA': 53, 'CO': 7, 'CT': 5, 'DE': 1,'FL': 27, 'GA': 14, 
'HI': 2, 'ID': 2, 'IL': 18, 'IN': 9, 'IA': 4, 'KS': 4, 'KY': 6, 'LA': 6, 'ME': 2, 'MD': 8,
'MA': 9, 'MI': 14, 'MN': 8, 'MS': 4, 'MO': 8, 'MT': 1, 'NE': 3, 'NV': 4, 'NH': 2, 'NJ': 12,
'NM': 3, 'NY': 27, 'NC': 13, 'ND': 1, 'OH': 16, 'OK': 5, 'OR': 5, 'PA': 18, 'RI': 2, 'SC': 7,
'SD': 1, 'TN': 9, 'TX': 36, 'UT': 4, 'VT': 1, 'VA': 11, 'WA': 10, 'WV': 3, 'WI': 8, 'WY': 1
}

In [16]:
def state_candidate(state = all, district = all):
    global candidate_df
    candidate_df = pd.DataFrame()
    global district_dict

    if state == all and district == all:
        for s, d in district_dict.items():
            time.sleep(1.5)
            for num in range(1, d + 1):
                district_scraper(str(s), str(num).zfill(2))
                time.sleep(1.5)
    
    elif district == all:
        for num in range(1, district_dict[state] + 1):
            district_scraper(str(state), str(num).zfill(2))
    
    elif state == all:
        for s, d in district_dict.items():
            try:   
                district_scraper(str(s), str(district).zfill(2))
            except:
                pass
    
    else:
        district_scraper(str(state), str(district).zfill(2))
    
    return candidate_df

In [17]:
#state_candidate(all, all).to_csv('../data/all_candidates.csv')

In [18]:
all_candidates = pd.read_csv('../data/all_candidates.csv')

In [19]:
all_candidates.shape

(892, 11)

In [20]:
all_candidates.groupby('state')['district'].max()

state
Alabama            7
Alaska             1
Arizona            9
Arkansas           4
California        53
Colorado           7
Connecticut        5
Delaware           1
Florida           27
Georgia           14
Hawaii             2
Idaho              2
Illinois          18
Indiana            9
Iowa               4
Kansas             4
Kentucky           6
Louisiana          6
Maine              2
Maryland           8
Massachusetts      9
Michigan          14
Minnesota          8
Mississippi        4
Missouri           8
Montana            1
Nebraska           3
Nevada             4
New Hampshire      2
New Jersey        12
New Mexico         3
New York          27
North Carolina    13
North Dakota       1
Ohio              16
Oklahoma           5
Oregon             5
Pennsylvania      18
Rhode Island       2
South Carolina     7
South Dakota       1
Tennessee          9
Texas             36
Utah               4
Vermont            1
Virginia          11
Washington        10
West Vi